In [15]:
# Import required libraries
import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.express as px
import requests
import numpy as np

In [11]:
#download the CSV file

URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'

def download(url, filename):
    response = requests.get(url)
    if response.status_code==200:
        with open(filename, 'wb') as f:
            f.write(response.content)
            

download(URL,'spacex_csv_file.csv')
spacex_df=pd.read_csv('spacex_csv_file.csv')


In [3]:
spacex_df.columns

Index(['Flight Number', 'Date', 'Time (UTC)', 'Booster Version', 'Launch Site',
       'Payload', 'Payload Mass (kg)', 'Orbit', 'Customer', 'Landing Outcome',
       'class', 'Lat', 'Long'],
      dtype='object')

In [14]:
booster = spacex_df.groupby('Booster Version')['class'].mean()
booster

Booster Version
F9 B4  B1039.2    0.0
F9 B4  B1040.2    0.0
F9 B4  B1041.2    0.0
F9 B4  B1043.2    0.0
F9 B4 B1039.1     1.0
F9 B4 B1040.1     1.0
F9 B4 B1041.1     1.0
F9 B4 B1042.1     1.0
F9 B4 B1043.1     1.0
F9 B4 B1044       0.0
F9 B4 B1045.1     1.0
F9 B5  B1046.1    1.0
F9 FT  B1021.2    1.0
F9 FT  B1029.2    1.0
F9 FT  B1031.2    1.0
F9 FT  B1032.2    0.0
F9 FT  B1035.2    1.0
F9 FT  B1036.2    0.0
F9 FT  B1038.2    0.0
F9 FT B1019       1.0
F9 FT B1020       0.0
F9 FT B1021.1     1.0
F9 FT B1022       1.0
F9 FT B1023.1     1.0
F9 FT B1024       0.0
F9 FT B1025.1     1.0
F9 FT B1026       1.0
F9 FT B1029.1     1.0
F9 FT B1030       0.0
F9 FT B1031.1     1.0
F9 FT B1032.1     1.0
F9 FT B1034       0.0
F9 FT B1035.1     1.0
F9 FT B1036.1     1.0
F9 FT B1037       0.0
F9 FT B1038.1     1.0
F9 v1.0  B0003    0.0
F9 v1.0  B0004    0.0
F9 v1.0  B0005    0.0
F9 v1.0  B0006    0.0
F9 v1.0  B0007    0.0
F9 v1.1           0.0
F9 v1.1  B1003    0.0
F9 v1.1 B1010     0.0
F9 v1.1 B1011   

In [16]:
spacex_df['Booster Version Category'] = np.select([spacex_df['Booster Version'].str.contains('B4'),
                        spacex_df['Booster Version'].str.contains('B5'),
                        spacex_df['Booster Version'].str.contains('FT'),
                        spacex_df['Booster Version'].str.contains('v1.0'),
                        spacex_df['Booster Version'].str.contains('v1.1')],
                       choicelist=['B4', 'B5', 'FT', 'v1.0','v1.1'])

In [22]:
# create a dash application
app = dash.Dash(__name__)

# build dash layout
app.layout = html.Div(
                        children= 
                                #Dashboard Heading
                                [html.H1('SpaceX Launch Records Dashboard',
                                       style={'textalign':'center',
                                              'color': '#503D36',
                                              'font-size': 20}),
                                html.Div([
                                        html.Div([
                                        #Launch Site dropdown menu
                                            dcc.Dropdown(id='site-dropdown',
                                                    options=[{'label':'All Sites', 'value':'All'},
                                                              {'label':'CCAFS LC-40', 'value':'CCAFS LC-40'},
                                                              {'label':'CCAFS SLC-40', 'value':'CCAFS SLC-40'},
                                                              {'label':'KSC LC-39A', 'value':'KSC LC-39A'},
                                                              {'label':'VAFB SLC-4', 'value':'VAFB SLC-4E'}],
                                                    value='All',
                                                    placeholder='Select a launch site here',
                                                    searchable=True
                                                    )
                                             ]),
                                        #Success Pie Chart
                                        html.Div(dcc.Graph(id='success-pie-chart'))
                                        ]),
                                 
                                html.Div([
                                        #Payload Range Slider heading
                                        html.Div(html.H2('Payload Range (kg)',
                                                           style={'textalign':'center',
                                                                  'color': '#503D36',
                                                                  'font-size': 20}
                                                        )
                                                ),
                                        #Payload Mass range slider
                                        html.Div(dcc.RangeSlider(id = 'payload-slider',
                                                             min=0, max=10000, step=1000,
                                                             marks={2500:'2500',5000:'5000',7500:'7500'},
                                                             value=[spacex_df['Payload Mass (kg)'].min(),spacex_df['Payload Mass (kg)'].max()]
                                                                )
                            
                                                ),
                                        html.Div(dcc.Graph(id='success-payload-scatter-chart'))
                                
                                        ])
    
                                ]
                    )



@app.callback(Output(component_id='success-pie-chart', component_property='figure'),
              Input(component_id='site-dropdown',component_property='value'))

def get_pie_chart(entered_site):
    
    
    if entered_site == 'All':
        
        filtered_df = spacex_df[spacex_df['class']==1]
        fig = px.pie(filtered_df, values='class', names='Launch Site', title='Total Number of successful Launches by Site')
        fig.update_traces(textinfo='value')
        #fig.show()
        return fig

    else:
        filtered_df = spacex_df[spacex_df['Launch Site']==entered_site]
        filtered_df=filtered_df.groupby(['Launch Site','class']).size().reset_index(name='class count')
        fig=px.pie(filtered_df,values='class count',names='class',title=f"Launch Success Rate for site: {entered_site}")        #fig2 = px.scatter(filtered_df, x='Payload Mass (kg)', y='class', color='Booster Version')
        return fig
    

@app.callback(Output(component_id='success-payload-scatter-chart',component_property='figure'),
                [Input(component_id='site-dropdown',component_property='value'),
                Input(component_id='payload-slider',component_property='value')])

def get_scatter_chart(entered_site,payload):
    filtered_df = spacex_df[spacex_df['Payload Mass (kg)'].between(payload[0],payload[1])]
    if entered_site=='All':
        fig=px.scatter(filtered_df,x='Payload Mass (kg)',y='class',color='Booster Version Category',title='Success vs Payload mass for all sites')
        return fig
    else:
        fig=px.scatter(filtered_df[filtered_df['Launch Site']==entered_site],x='Payload Mass (kg)',y='class',color='Booster Version Category',title=f"Success vs Payload mass for site: {entered_site}")
        return  fig

# Run the app
if __name__ == '__main__':
    app.run_server()

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [16/Jun/2023 20:53:36] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [16/Jun/2023 20:53:36] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [16/Jun/2023 20:53:36] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [16/Jun/2023 20:53:36] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 200 -
127.0.0.1 - - [16/Jun/2023 20:53:36] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 200 -
127.0.0.1 - - [16/Jun/2023 20:53:36] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 200 -
127.0.0.1 - - [16/Jun/2023 20:53:36] "GET /_dash-component-suites/dash/dcc/async-slider.js HTTP/1.1" 200 -
127.0.0.1 - - [16/Jun/2023 20:53:36] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [16/Jun/2023 20:53:36] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [16/Jun/2023 20:53:47] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [16/Jun/2023 20:54:01] "POST /_dash-